In [7]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import q2_script

In [8]:
x_train, y_train, x_test, y_test = q2_script.read_dataset()

x_val = x_train[0:10000]
y_val = y_train[0:10000]

x_train = x_train[10000:]
y_train = y_train[10000:]

In [9]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (50000, 784)
y_train shape: (50000, 10)
x_val shape: (10000, 784)
y_val shape: (10000, 10)
x_test shape: (10000, 784)
y_test shape: (10000, 10)


In [10]:
# logistic regression model

class LogisticRegression:
    def __init__(self, x_train, y_train, x_val, y_val, x_test, y_test, weight_initilisation):
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.x_test = x_test
        self.y_test = y_test
        self.weights = self.initialize_weights(weight_initilisation)
    
    def initialize_weights(self, weight_initilisation):
        if weight_initilisation == "normal" or weight_initilisation == "gaussian":
            # normal distribution with mean 0 and variance 1
            self.weights = np.random.normal(loc=0.0, scale=1.0, size=(10, 784))
        elif weight_initilisation == "zeros":
            self.weights = np.zeros((10, 784))
        elif weight_initilisation == "uniform":
            self.weights = np.random.uniform(low=-1, high=1, size=(10, 784))
        else:
            raise Exception("Invalid weight initilisation method, choose from: \"normal\", \"zeros\" or \"uniform\"")
        return self.weights
    
    def softmax(self, z):
        # z is a vector of shape (10, 1)
        return np.exp(z) / np.sum(np.exp(z), axis=0)
    
    
    def forward(self, x):
        # x is a vector of shape (784, 1)
        # weights is a matrix of shape (10, 784)
        # z is a vector of shape (10, 1)
        z = np.matmul(self.weights, x)
        return self.softmax(z)
    
    def train(self, epochs, lr, batch_size, L2_regularisation_coeff):
        # epochs: number of iterations
        # lr: learning rate
        # batch_size: size of mini-batch
        # L2_regularisation_coeff: lambda
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1} of {epochs}")
            for i in range(0, len(self.x_train), batch_size):
                x_batch = self.x_train[i:i + batch_size]
                y_batch = self.y_train[i:i + batch_size]
                self.update_weights(x_batch, y_batch, lr, L2_regularisation_coeff)
            self.evaluate()
    
    # I was here    
    def update_weights(self, x_batch, y_batch, lr, L2_regularisation_coeff):
        # x_batch is a matrix of shape (batch_size, 784)
        # y_batch is a matrix of shape (batch_size, 10)
        # weights is a matrix of shape (10, 784)
        # y_pred is a matrix of shape (batch_size, 10)
        # y_batch is a matrix of shape (batch_size, 10)
        # x_batch.T is a matrix of shape (784, batch_size)
        # grad is a matrix of shape (10, 784)
        # L2_regularisation_coeff is a scalar
        y_pred = self.forward(x_batch.T)
        grad = np.matmul((y_pred - y_batch.T), x_batch) / len(x_batch) + L2_regularisation_coeff * self.weights
        self.weights -= lr * grad.T
    
    def evaluate(self):
        # x_val is a matrix of shape (10000, 784)
        # y_val is a matrix of shape (10000, 10)
        # weights is a matrix of shape (10, 784)
        # y_pred is a matrix of shape (10000, 10)
        # y_val is a matrix of shape (10000, 10)
        # correct is a scalar
        y_pred = self.forward(self.x_val.T)
        correct = np.sum(np.argmax(y_pred, axis=0) == np.argmax(self.y_val, axis=1))
        print(f"Validation accuracy: {correct / len(self.x_val)}")
    
    def test(self):
        # x_test is a matrix of shape (10000, 784)
        # y_test is a matrix of shape (10000, 10)
        # weights is a matrix of shape (10, 784)
        # y_pred is a matrix of shape (10000, 10)
        # y_test is a matrix of shape (10000, 10)
        # correct is a scalar
        y_pred = self.forward(self.x_test.T)
        correct = np.sum(np.argmax(y_pred, axis=0) == np.argmax(self.y_test, axis=1))
        print(f"Test accuracy: {correct / len(self.x_test)}")

        

In [12]:
logistic_regression = LogisticRegression(x_train, y_train, x_val, y_val, x_test, y_test, "normal")
logistic_regression.train(epochs=100, lr=0.0005, batch_size=200, L2_regularisation_coeff=0.0001)

Epoch 1 of 100


C:\Users\NSagi\AppData\Local\Temp\ipykernel_12572\3816643186.py:27: RuntimeWarning: overflow encountered in exp
  return np.exp(z) / np.sum(np.exp(z), axis=0)
C:\Users\NSagi\AppData\Local\Temp\ipykernel_12572\3816643186.py:27: RuntimeWarning: invalid value encountered in divide
  return np.exp(z) / np.sum(np.exp(z), axis=0)


ValueError: operands could not be broadcast together with shapes (10,784) (784,10) (10,784) 